In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader as pdr
from datetime import datetime
%matplotlib inline


In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [4]:
#Import data
def stock_data(ticker, start, end):
    try:
        data = yf.download(ticker,start, end, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error: {e}")
    return data

In [83]:
#Import S&P500
prices = stock_data(['SPY','GLD'], "2000-08-01", "2026-01-31")
prices.head(20)


[*********************100%***********************]  2 of 2 completed


Ticker,GLD,SPY
Date,,
2000-08-01,NaN,91.077049
2000-08-02,NaN,91.532005
2000-08-03,NaN,92.165070
2000-08-04,NaN,92.659615
2000-08-07,NaN,93.767456
2000-08-08,NaN,94.123528
2000-08-09,NaN,93.332199
2000-08-10,NaN,92.877251
2000-08-11,NaN,93.312431


In [84]:
prices.dropna()

Ticker,GLD,SPY
Date,,
2004-11-18,44.380001,80.169014
2004-11-19,44.779999,79.277824
2004-11-22,44.950001,79.655930
2004-11-23,44.750000,79.777466
2004-11-24,45.049999,79.966522
...,...,...
2026-01-26,464.700012,692.729980
2026-01-27,476.100006,695.489990
2026-01-28,494.559998,695.419983


In [85]:
class RotationStrategy:
    
    def __init__(self,
                prices =pd.DataFrame):
        """ SPY and GLD daily prices data  """
        self.prices =prices
        
    
    def rotation_strategy(self, ma_window=6, target_vol = 0.12, capital = 1000):
        """calculating the strategy"""

        #Resampling to get the price of the last business day of the month
        monthly_prices = prices.resample("ME").last()
        monthly_prices.dropna(inplace=True)
        
        #Track the ratio of Gold prices to S&P500
        monthly_prices["price_ratio"] = monthly_prices['GLD']/monthly_prices['SPY']

        #Moving Average of the ratio
        monthly_prices['ratio_moving_avg']= monthly_prices['price_ratio'].rolling(window= ma_window).mean()
        monthly_prices.dropna(inplace=True)

        #Generating signal
        signal = (monthly_prices['price_ratio'] < monthly_prices['ratio_moving_avg']).astype(int)
        position= signal.shift(1).dropna()

        #Calculating monthly returns
        monthly_returns = monthly_prices.pct_change().loc[position.index]
    
        #Calculating return startegy and equity curve
        strategy_returns = (position*monthly_returns['SPY'] + (1-position) * monthly_returns['GLD'])
        equity_curve = (1+strategy_returns).cumprod()
          
        #calculating returns for buy and hold strategy
        gold_bnh = (1+monthly_returns['GLD']).cumprod()
        spy_bnh = (1+monthly_returns['SPY']).cumprod()

        #Plotting the curves
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve,
                         mode='lines', name = 'Equity Curve'))
        fig.add_trace(go.Scatter(x=gold_bnh.index, y = gold_bnh,
                         mode='lines', name = 'Gold Buy and Hold'))
        fig.add_trace(go.Scatter(x=spy_bnh.index, y = spy_bnh,
                         mode='lines', name = 'S&P Buy and hold'))

        fig.update_layout(autosize = False,
                          width = 1000, height = 400,title='Equity Curve', 
                  xaxis_title='Date', yaxis_title ='Performance',
                  template="plotly_white",
            hovermode="x unified")
        fig.show()

        #Adding turnover and rebalance detection
        rebalance = position.diff().abs() == 1

        #assuming transaction cost as 10 bps
        cost = 0.001
        strategy_returns[rebalance] -= cost

    
        #Trade log to show the months when we either buy SPY or buy GLD
        position_change = position.diff()

        trade_log = []

        for date, change in position_change.dropna().items():
            if change == 1:
                trade_log.append({
                    "Date": date,
                    "Action": "Buy",
                    "Asset": "SPY",
                    "Price": monthly_prices.loc[date, "SPY"]
                })
                trade_log.append({
                    "Date": date,
                    "Action": "Sell",
                    "Asset": "GOLD",
                    "Price": monthly_prices.loc[date, "GLD"] })
            
            elif change == -1:
                trade_log.append({
                    "Date": date,
                    "Action": "Buy",
                    "Asset": "GOLD",
                    "Price": monthly_prices.loc[date, "GLD"] })
                
                trade_log.append({
                    "Date": date,
                    "Action": "Sell",
                    "Asset": "SPY",
                    "Price": monthly_prices.loc[date, "SPY"] })
                
        trade_log = pd.DataFrame(trade_log)

        trade_log["Side"] = trade_log["Action"].map({"Buy": 1,"Sell":-1})
        
        print (trade_log)


        #Plotting ratio and ratio_ma
        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(x=monthly_prices.index, y = monthly_prices['price_ratio'],
                         mode='lines', name = 'Price Ratio'))
        fig1.add_trace(go.Scatter(x=monthly_prices.index, y = monthly_prices['ratio_moving_avg'],
                         mode='lines', name = 'Ratio moving Average'))
        

        fig1.update_layout(autosize = False,
                          width = 1000, height = 400, title='Price ratio and ratio Moving Average', 
                  xaxis_title='Date',
                  template="plotly_white",
            hovermode="x unified")
        fig1.show()


        #Volatility Targeting
        spy_vol = monthly_returns['SPY'].rolling(12).std() * np.sqrt(12)
        gld_vol = monthly_returns['GLD'].rolling(12).std() * np.sqrt(12)

        active_vol = position * spy_vol + (1- position) * gld_vol

        leverage = target_vol/active_vol
        leverage = leverage.clip(0, 2.0) # cap leverage at 2x

        #Volatility scaled strategy returns
        strategy_returns_vt = leverage * (position * monthly_returns['SPY'] + (1- position)* monthly_returns['GLD'])
        equity_vt = (1+ strategy_returns_vt).cumprod()

        trade_log['Leverage'] = trade_log['Date'].map(leverage)
        trade_log['Notional'] = capital * trade_log['Leverage']
        trade_log['Units'] = trade_log['Notional']/trade_log['Price']
        trade_log['Units'] *= trade_log['Side']

        strategy_returns_vt[rebalance] -= cost

        #Plotting equity curve with and without volatility targeting
        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve,
                         mode='lines', name = 'Equity Curve'))
        fig2.add_trace(go.Scatter(x=equity_vt.index, y = equity_vt,
                         mode='lines', name = 'Equity curve with volatility Targeting'))
    
        fig2.update_layout(autosize = False,
                          width = 1000, height = 400, title='Equity Curve', 
                  xaxis_title='Date', yaxis_title ='Comparison',
                  template="plotly_white",
            hovermode="x unified")
        fig2.show()

        return strategy_returns, strategy_returns_vt, monthly_returns

    def performance_metrics(self, strategy_returns, freq = 12):
        """ Performance metrics"""
        strategy_returns  = strategy_returns.dropna()
        equity = (1+ strategy_returns).cumprod()

        total_periods = len(strategy_returns)
        years = total_periods /freq

        cagr = equity.iloc[-1]**(1/years) - 1
        vol =  strategy_returns.std() * np.sqrt(freq)
        sharpe = cagr/vol if vol!= 0 else np.nan

        downside = strategy_returns[strategy_returns < 0].std() * np.sqrt(freq)
        sortino = cagr / downside if downside !=0 else np.nan

        running_max = equity.cummax()
        drawdown = equity/running_max -1
        max_dd = drawdown.min()

        calmar = cagr/abs(max_dd) if max_dd != 0 else np.nan

        win_rate = (strategy_returns > 0).mean()
        avg_monthly = strategy_returns.mean()

        return pd.Series({
            "CAGR": cagr,
            "Volatility": vol,
            "Sharpe": sharpe,
            "Sortino": sortino,
            "Max drawdown": max_dd,
            "Calmar": calmar,
            "Win Rate": win_rate,
            "Avg Monthly Return": avg_monthly
              })










In [86]:
strategy = RotationStrategy(prices)

In [87]:
strategy_returns, strategy_vt, monthly_returns = strategy.rotation_strategy(6, 0.12, 1000)

          Date Action Asset       Price  Side
0   2005-06-30    Buy   SPY   81.491364     1
1   2005-06-30   Sell  GOLD   43.439999    -1
2   2005-07-31    Buy  GOLD   42.820000     1
3   2005-07-31   Sell   SPY   84.609367    -1
4   2005-08-31    Buy   SPY   83.816162     1
..         ...    ...   ...         ...   ...
115 2025-02-28   Sell   SPY  587.283386    -1
116 2025-08-31    Buy   SPY  641.371399     1
117 2025-08-31   Sell  GOLD  318.070007    -1
118 2025-10-31    Buy  GOLD  368.119995     1
119 2025-10-31   Sell   SPY  680.050537    -1

[120 rows x 5 columns]


In [88]:
performance_vt = strategy.performance_metrics(strategy_vt, 12)
performance_vt

CAGR                  0.123162
Volatility            0.127904
Sharpe                0.962925
Sortino               1.829667
Max drawdown         -0.292524
Calmar                0.421032
Win Rate              0.605042
Avg Monthly Return    0.010399
dtype: float64

In [89]:
performance = strategy.performance_metrics(strategy_returns, 12)
performance

CAGR                  0.119474
Volatility            0.156682
Sharpe                0.762526
Sortino               1.192008
Max drawdown         -0.433291
Calmar                0.275736
Win Rate              0.602410
Avg Monthly Return    0.010465
dtype: float64

In [90]:
monthly_returns

Ticker,GLD,SPY,price_ratio,ratio_moving_avg
Date,,,,
2005-05-31,-0.039216,0.032225,-0.069210,-0.016960
2005-06-30,0.042977,0.001515,0.041400,-0.000355
2005-07-31,-0.014273,0.038262,-0.050598,-0.006478
2005-08-31,0.013545,-0.009375,0.023137,-0.004472
2005-09-30,0.076037,0.008026,0.067470,0.006252
...,...,...,...,...
2025-09-30,0.117584,0.035620,0.079145,0.005084
2025-10-31,0.035587,0.023837,0.011476,-0.003649
2025-11-30,0.053678,0.001950,0.051628,0.016176


In [91]:
perfomance_spy = strategy.performance_metrics(monthly_returns['SPY'], 12)

In [92]:
summary = pd.concat([performance, performance_vt, perfomance_spy], axis =1)
summary.columns = ["No Vol Target", "Vol Targeted", "SPY"]
summary

,No Vol Target,Vol Targeted,SPY
CAGR,0.119474,0.123162,0.110364
Volatility,0.156682,0.127904,0.148536
Sharpe,0.762526,0.962925,0.743013
Sortino,1.192008,1.829667,0.993027
Max drawdown,-0.433291,-0.292524,-0.507848
Calmar,0.275736,0.421032,0.217317
Win Rate,0.602410,0.605042,0.674699
Avg Monthly Return,0.010465,0.010399,0.009688
